In [15]:
this_file_name = 'sketching_configuration_mapping.ipynb'
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(this_file_name), '../../src/') ))

from configuration_space_mapping import *
from utils import *

# The idea

First, retrieve all scene objects' configuration and position. 

In [16]:
client_id = connect_2_sim()
test_connection(client_id)

Connected to remote API server
Number of objects in the scene:  42


In [17]:
scene_objects = ['Cuboid_0','Cuboid_1','Cuboid_2',
                 'Cuboid_3','Cuboid_4','dr20']

In [18]:
listazinha = get_scene_objects_info(client_id, scene_objects)

In [19]:
for element in listazinha:
    a = round(element['object_position'][0],2)
    b = round(element['object_position'][1],2)
    c = round(element['object_position'][2],2)
    print([a,b,c])

[-1.72, 0.25, 0.25]
[-0.35, 1.5, 0.5]
[-1.52, -1.22, 0.5]
[-0.15, -0.1, 0.25]
[1.4, -0.15, 0.5]
[-2.02, 1.73, 0.15]


In [20]:
info_list, robot_information = split_robot_from_info_list(listazinha,'dr20')

In [21]:
for element in info_list:
    print(element['object_name'])

Cuboid_0
Cuboid_1
Cuboid_2
Cuboid_3
Cuboid_4


we know there are 4 horizontal normal vectors. Two of them point to X and Y. The other two point to -X and -Y. 

How to know where X is pointing? Well, it is given by the Z Euler vector. 

Where does Y point to? to X+90º. And what about -X and -Y? We just need to add 180º to X and Y.

# Creating Configuration Map

1. Step
To orient the robot so its euler angles will be 0,0,0.
2. Step
For each obstacle, do the following:
-  

In [44]:
handle_dr20 = get_object_handle(client_id,'dr20')
euler_orientation = (0,0,0)
sim.simxSetObjectOrientation(client_id,handle_dr20,-1,euler_orientation,sim.simx_opmode_blocking)

0

In [45]:
robot_pos, robot_ang = get_configuration(client_id, handle_dr20)
180*robot_ang[2]/3.14

0.03242413780301999

In [54]:
get_normals_orientation(0)

(0, 1.570796326795, 3.14159265359, 4.7123889803850005)

In [50]:
listazinha[0]['object_orientation'][2]+3.14/2

1.57